In [1]:
import os
import psycopg2
from sqlalchemy import create_engine
from dotenv import load_dotenv
import pandas as pd

In [2]:
# Parâmetros de conexão com o PostgreSQL
db_host = 'localhost'
db_port = os.getenv('PREFECT_DB_PORT')
db_database = os.getenv('PREFECT_DB_DATABASE')
db_user = os.getenv('PREFECT_DB_USER')
db_password = os.getenv('PREFECT_DB_PASSWORD')

In [3]:
try:
    # Testar conexão antes de prosseguir
    test_conn = psycopg2.connect(
        host=db_host,
        port=db_port,
        database=db_database,
        user=db_user,
        password=db_password
    )
    print("Conexão com o PostgreSQL estabelecida com sucesso!")
    test_conn.close()
except Exception as e:
    print(f"Erro ao estabelecer conexão com o PostgreSQL: {e}")

Conexão com o PostgreSQL estabelecida com sucesso!


In [4]:
engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_database}')


In [6]:

query = """-- Listar Tabelas
SELECT table_schema, table_name, 'table' AS type
FROM information_schema.tables
WHERE --table_schema IN ('bronze', 'silver', 'gold')
  --AND 
  table_type = 'BASE TABLE'

UNION

-- Listar Views
SELECT table_schema, table_name, 'view' AS type
FROM information_schema.views
--WHERE table_schema IN ('bronze', 'silver', 'gold')

UNION

-- Listar Views Materializadas
SELECT schemaname, matviewname AS table_name, 'materialized view' AS type
FROM pg_matviews
--WHERE schemaname IN ('bronze', 'silver', 'gold')
ORDER BY table_schema, table_name;
"""
available_tables = pd.read_sql(query, con=engine)
available_tables

,table_schema,table_name,type
0,hdb_catalog,event_invocation_logs,table
1,hdb_catalog,event_log,table
2,hdb_catalog,event_triggers,table
3,hdb_catalog,hdb_action,table
4,hdb_catalog,hdb_action_log,table
...,...,...,...
235,public,task_run,table
236,public,task_run_artifact,table
237,public,task_run_state,table
238,public,tenant,table


In [8]:
pd.read_sql("SELECT * FROM public.flow_run", con=engine)

,id,tenant_id,created,flow_id,parameters,scheduled_start_time,auto_scheduled,heartbeat,start_time,end_time,...,state_start_time,serialized_state,name,context,times_resurrected,updated,idempotency_key,agent_id,labels,run_config
0,dee4eaf2-0675-4cd7-a351-0c9241f3064d,54163aa8-7d17-489e-adbf-f4b110ee06bd,2025-01-23 06:02:02.482010+00:00,04910e29-18ec-4abe-8289-49ef23639d1b,"{'materialization_mode': 'dev', 'materialize_a...",2025-01-23 06:11:00+00:00,True,NaT,NaT,NaT,...,2025-01-23 06:11:00+00:00,"{'type': 'Scheduled', '_result': {'type': 'NoR...",spotted-octopus,{},0,2025-01-23 06:07:03.560134+00:00,auto-scheduled:2025-01-23T06:11:00+00:00:3ce13...,None,[pipelines],"{'env': None, 'type': 'LocalRun', 'labels': ['..."
1,e8fc2b10-a1ab-4135-b25d-6e3e58ef36a8,54163aa8-7d17-489e-adbf-f4b110ee06bd,2025-01-23 06:07:03.518366+00:00,04910e29-18ec-4abe-8289-49ef23639d1b,"{'materialization_mode': 'dev', 'materialize_a...",2025-01-23 06:17:00+00:00,True,NaT,NaT,NaT,...,2025-01-23 06:17:00+00:00,"{'type': 'Scheduled', '_result': {'type': 'NoR...",careful-centipede,{},0,2025-01-23 06:07:03.560134+00:00,auto-scheduled:2025-01-23T06:17:00+00:00:3ce13...,None,[pipelines],"{'env': None, 'type': 'LocalRun', 'labels': ['..."
2,c33a5d49-df56-4e1d-9e1f-76cdd07da47e,54163aa8-7d17-489e-adbf-f4b110ee06bd,2025-01-23 06:02:02.465302+00:00,04910e29-18ec-4abe-8289-49ef23639d1b,"{'materialization_mode': 'dev', 'materialize_a...",2025-01-23 06:03:00+00:00,True,2025-01-23 06:03:02.943164+00:00,2025-01-23 06:03:02.463632+00:00,2025-01-23 06:03:04.550764+00:00,...,NaT,"{'type': 'Success', '_result': {'type': 'Resul...",crazy-stork,{},0,2025-01-23 06:03:04.554490+00:00,auto-scheduled:2025-01-23T06:03:00+00:00:3ce13...,a7f215fe-5968-4551-81bc-12c1a0bd7c5a,[pipelines],"{'env': None, 'type': 'LocalRun', 'labels': ['..."
3,f73ed390-92e8-40c8-8b6e-b43198c7aae8,54163aa8-7d17-489e-adbf-f4b110ee06bd,2025-01-23 06:02:02.475294+00:00,04910e29-18ec-4abe-8289-49ef23639d1b,"{'materialization_mode': 'dev', 'materialize_a...",2025-01-23 06:05:00+00:00,True,2025-01-23 06:05:01.383581+00:00,2025-01-23 06:05:00.931347+00:00,2025-01-23 06:05:03.377234+00:00,...,NaT,"{'type': 'Success', '_result': {'type': 'Resul...",nano-woodpecker,{},0,2025-01-23 06:05:03.382699+00:00,auto-scheduled:2025-01-23T06:05:00+00:00:3ce13...,a7f215fe-5968-4551-81bc-12c1a0bd7c5a,[pipelines],"{'env': None, 'type': 'LocalRun', 'labels': ['..."
4,55b6dd4c-b874-440f-8b28-fca09f10fa6d,54163aa8-7d17-489e-adbf-f4b110ee06bd,2025-01-23 06:07:03.519889+00:00,04910e29-18ec-4abe-8289-49ef23639d1b,"{'materialization_mode': 'dev', 'materialize_a...",2025-01-23 06:15:00+00:00,True,NaT,NaT,NaT,...,2025-01-23 06:15:00+00:00,"{'type': 'Scheduled', '_result': {'type': 'NoR...",esoteric-ant,{},0,2025-01-23 06:07:03.560134+00:00,auto-scheduled:2025-01-23T06:15:00+00:00:3ce13...,None,[pipelines],"{'env': None, 'type': 'LocalRun', 'labels': ['..."
5,8ee008e1-04ce-41f8-956e-ace9c9a6e660,54163aa8-7d17-489e-adbf-f4b110ee06bd,2025-01-23 06:07:03.527283+00:00,04910e29-18ec-4abe-8289-49ef23639d1b,"{'materialization_mode': 'dev', 'materialize_a...",2025-01-23 06:16:00+00:00,True,NaT,NaT,NaT,...,2025-01-23 06:16:00+00:00,"{'type': 'Scheduled', '_result': {'type': 'NoR...",premium-petrel,{},0,2025-01-23 06:07:03.560134+00:00,auto-scheduled:2025-01-23T06:16:00+00:00:3ce13...,None,[pipelines],"{'env': None, 'type': 'LocalRun', 'labels': ['..."
6,2edaf360-51cb-478f-9bdf-d7716be62815,54163aa8-7d17-489e-adbf-f4b110ee06bd,2025-01-23 06:02:02.475593+00:00,04910e29-18ec-4abe-8289-49ef23639d1b,"{'materialization_mode': 'dev', 'materialize_a...",2025-01-23 06:06:00+00:00,True,2025-01-23 06:06:02.709747+00:00,2025-01-23 06:06:02.223240+00:00,2025-01-23 06:06:04.355404+00:00,...,NaT,"{'type': 'Success', '_result': {'type': 'Resul...",ruddy-gaur,{},0,2025-01-23 06:06:04.359773+00:00,auto-scheduled:2025-01-23T06:06:00+00:00:3ce13...,a7f215fe-5968-4551-81bc-12c1a0bd7c5a,[pipelines],"{'env': None, 'type': 'LocalRun', 'labels': ['..."
7,7b03f578-94ff-4c84-